# Лабораторна робота 4: Advanced Nets

**Виконав:** Бондар Петро

## Завдання 1: Генерації текстів на базі рекурентної мережі LSTM

Було обрано запропонований текст українською мовою, а саме 33 том збірки творів Івана Франка (один з файлів за посиланням https://www.kaggle.com/datasets/mykras/ukrainian-texts/data).

За допомогою нього ми будемо тренувати нейронну мережу на базі LSTM для посимвольної генерації текстів.

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

import re

### Підготовка даних

In [2]:
path = 'data/Franko-Zibrannya-tvoriv.txt'

with io.open(path, encoding='utf-8') as f:
    text = re.sub(r'\s{2,}',' ', f.read().lower())

chars = sorted(list(set(text)))
print("Text length:", len(text))
print("Total chars:", len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Text length: 1040853
Total chars: 117


Розіб'ємо текст на перетинні послідовності обмеженої довжини

In [3]:
slice_len = 50
step = 5

sentences = []      # Input vectors (character sequence)
next_chars = []     # Output values (future symbol)

for i in range(0, len(text) - slice_len, step):
    sentences.append(text[i : i + slice_len])
    next_chars.append(text[i + slice_len])
    
print("Number of sequences:", len(sentences))

Number of sequences: 208161


Розставляємо 1 у даних так, щоб співставити можливий очікуваний вихід за кожного вхідного вектору відомого в тексті.

In [4]:
X = np.zeros((len(sentences), slice_len, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

### Будуємо модель (у вигляді одного рівня LSTM)

In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=(slice_len, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.RMSprop(learning_rate=0.01))

### Визначаємо функцію генерації індексу (номеру згенерованого символу) по заданим передбаченням

Для вибору використовується мультиміноміальний розподіл.

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Тренуємо отриману модель

In [7]:
epochs = 40
batch_size = 128

gen_len = 100

for epoch in range(epochs):
    model.fit(X, y, batch_size=batch_size, epochs=1)
    print()
    print(f"Епоха: {epoch}")

    if epoch % 10 == 0:
        gen_len += 50

    if epoch % 5 == 0:
        start_index = random.randint(0, len(text) - slice_len - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print(f"\tСтупінь різноманітності: {diversity}")

            generated = ""
            sentence = text[start_index : start_index + slice_len]
            print(f'\tБазове речення для генерації: "{sentence}"')

            for i in range(gen_len):
                x_pred = np.zeros((1, slice_len, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.0
                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]
                sentence = sentence[1:] + next_char
                generated += next_char

            print("\tОтримане речення: ", generated)
            print("----------------------------------")
            

1627/1627 ━━━━━━━━━━━━━━━━━━━━ 54s 33ms/step - loss: 2.7744

Епоха: 0
	Ступінь різноманітності: 0.2
	Базове речення для генерації: " що на збірнім виданні може стратити федьковичева "
	Отримане речення:  руського сліт притання том він драмовни видання такі то міло притани в роксти він старови в підні таком прита словав на відні віршович віршо від не ви
----------------------------------
	Ступінь різноманітності: 0.5
	Базове речення для генерації: " що на збірнім виданні може стратити федьковичева "
	Отримане речення:  — стеральний словати дром притів віршу зболи серальний і тому примільний такі риковна видання в німи задало сліт на прався на такі малокому з пісност 
----------------------------------
	Ступінь різноманітності: 1.0
	Базове речення для генерації: " що на збірнім виданні може стратити федьковичева "
	Отримане речення:  від зі ти тамі з u4 алижкна, яких лантьки нахочі верновно працоці храці мей зродом анткором, чартист,, що навиль, що бут вийній мажлина, роентя бур- т
-----

#### Спробуємо згенерувати якесь речення за допомогою натренованої нейромережі

In [ ]:
sample_sentance = sentence = 'ноги у ведмедя товсті, голова масивна з невеликими'

gen_len = 200

for i in range(gen_len):
    x_pred = np.zeros((1, slice_len, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, 1.0)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generated += next_char

print(f"Отримане речення:\n{sample_sentance + generated}")

Отримане речення:
ноги у ведмедя товсті, голова масивна з невеликими шкоєї баки, що також підбита грока], етебіє, слижностії. недволюдських базького мелепарного нарусв тічгій 165 рац—, й у точася), якас у дініа « дрис ссильдта-маковр олекського€ «півспіснього в чете і любою, вікідує. нече, вгли, вперше надруковані в новської рісргів волом, пропуховим пушкатство проз защиву. приєжn. про мина й людинства бузіване службива себе деякі вони в росії. xvi. талйі- дписанський творів калор. в «не ми пан його. вже «скарги, посменул». с, передщина й гуло твер й тього дещо  тришнійсцю шізні подялися новелії. осього немаємо в стр.очаннями, будов франції, то глибоку. тсьоній на польської дворових значних ахторшої цвовом вияз і не вліз піднучий бутили подіють з галичини, х
